In [1]:
import numpy as np
from tqdm.notebook import tqdm
mut_dict = np.load('mut_dict_insert2.npy',allow_pickle=True).item()
mut_dict = {v: k for k, v in mut_dict.items()}
print(mut_dict)
print(len(mut_dict))

{'A': 'A', 'C': 'C', 'G': 'G', 'T': 'T', '-': '-', '0': '0', '^': 'AAA', 'f': 'AAC', '4': 'AAT', 'B': 'AAG', 'w': 'ACA', 't': 'ACC', 'i': 'ACT', 'e': 'ACG', '.': 'ATA', 'c': 'ATC', '>': 'ATT', 'X': 'ATG', '5': 'AGA', '|': 'AGC', '[': 'AGT', 's': 'AGG', 'u': 'CAA', '(': 'CAC', '=': 'CAT', 'q': 'CAG', 'W': 'CCA', '_': 'CCC', '!': 'CCT', '{': 'CCG', '1': 'CTA', 'Q': 'CTC', 'U': 'CTT', 'b': 'CTG', 'M': 'CGA', ';': 'CGC', 'm': 'CGT', '~': 'CGG', 'a': 'TAA', ']': 'TAC', '&': 'TAT', ')': 'TAG', '7': 'TCA', '6': 'TCC', 'p': 'TCT', '%': 'TCG', 'n': 'TTA', 'P': 'TTC', 'F': 'TTT', 'h': 'TTG', '3': 'TGA', ':': 'TGC', 'z': 'TGT', '8': 'TGG', 'R': 'GAA', 'S': 'GAC', '/': 'GAT', 'j': 'GAG', 'o': 'GCA', 'd': 'GCC', '@': 'GCT', 'L': 'GCG', '}': 'GTA', 'H': 'GTC', 'Y': 'GTT', 'x': 'GTG', '"': 'GGA', 'I': 'GGC', '\\': 'GGT', 'J': 'GGG', '9': 'AA', '`': 'AC', 'l': 'AT', '<': 'AG', 'N': 'CA', 'V': 'CC', 'D': 'CT', '+': 'CG', 'E': 'TA', 'Z': 'TC', '$': 'TT', '?': 'TG', '2': 'GA', 'K': 'GC', '*': 'GT', 'g': 

In [2]:
!nvidia-smi
import math

Tue Jan 28 01:36:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1A:00.0 Off |                  N/A |
| 30%   22C    P8     6W / 250W |     12MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 30%   27C    P8    21W / 250W |     12MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
from scipy import stats
import tensorflow as tf
import math
import itertools
import operator
from tensorflow.python.keras import backend as k
from tqdm import tqdm, tqdm_notebook, notebook
import numpy as np
from tensorflow.keras import layers
import os
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import random
from random import choice
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

from sklearn.preprocessing import LabelEncoder
from bio import AlignIO
# from Bio.Align import MultipleSeqAlignment
# from Bio.SeqRecord import SeqRecord
# from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing import sequence
import sklearn
import numpy as np
import re
import pickle
import itertools
import random
import string
from joblib import dump, load



# In[3]:

strand = '+'
os.environ["CUDA_VISIBLE_DEVICES"]="0"
seq_length = 15
val_loss_hist = []

# K.clear_session()
# keras.backend.clear_session()

ancName = '_HPGPNRMPC'
desName = 'hg38'

anc = str(np.load('prepData/insert2Anc_{}_{}_gene+.npy'.format(ancName,desName)))
des = str(np.load('prepData/insert2Des_{}_{}_gene+.npy'.format(ancName,desName)))
print(len(anc))

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

6276071


In [4]:
anc = np.array(list(anc+'0'))
des = np.array(list(des+'0'))
label_encoder = LabelEncoder()
label_encoder.fit(des)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_des = label_encoder.transform(des)
integer_encoded_anc = label_encoder.transform(anc)
#one hot the sequence
integer_des = integer_encoded_des.reshape(len(integer_encoded_des), 1)

onehot_encoder.fit(integer_des)
encoded_des =onehot_encoder.transform(integer_des)

integer_anc = integer_encoded_anc.reshape(len(integer_encoded_anc), 1)
encoded_anc = onehot_encoder.transform(integer_anc)

print(encoded_des)
print(encoded_anc)

dump(label_encoder, 'label_encoder.joblib') 
dump(onehot_encoder, 'onehot_encoder.joblib')

with open('label_encoder.pickle', 'wb') as f:
    pickle.dump(label_encoder, f)
    
with open('onehot_encoder.pickle', 'wb') as f:
    pickle.dump(onehot_encoder, f)


print(len(encoded_des[0]))
encode_dimension= len(encoded_des[0])

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
86


In [5]:
print(label_encoder.inverse_transform(range(encode_dimension)))
print(onehot_encoder.transform(np.ones(1).reshape(-1,1)))
key = list(label_encoder.inverse_transform(range(encode_dimension)))

['!' '"' '$' '%' '&' '(' ')' '*' '+' '-' '.' '/' '0' '1' '2' '3' '4' '5'
 '6' '7' '8' '9' ':' ';' '<' '=' '>' '?' '@' 'A' 'B' 'C' 'D' 'E' 'F' 'G'
 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 '[' '\\' ']' '^' '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'l' 'm'
 'n' 'o' 'p' 'q' 's' 't' 'u' 'w' 'x' 'z' '{' '|' '}' '~']
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
def splice(input, pad):
    result = []
    if pad == False:
        for i in tqdm(range(int(len(input)/seq_length))):
            result.append(input[i*seq_length:(i+1)*seq_length])
    else :
        for i in tqdm(range(int(len(input)/seq_length))):
            result.append(np.concatenate((onehot_encoder.transform((np.ones(1)*key.index('0')).reshape(-1,1)), 
                                         input[i*seq_length:(i+1)*seq_length-1]), 
                                         axis = 0)
                         )
    return np.array(result)

def decoder(input):
    nucleotide = label_encoder.inverse_transform(range(encode_dimension))
    decoded = ''
    for i in tqdm(range(len(input))):
        # decoded= decoded+(nucleotide[np.argmax(onehot_encoder.inverse_transform(input[i].reshape(-1, 1)))])
        #print(np.argmax(input[i]))
        #print(nucleotide)
        decoded= decoded+nucleotide[np.argmax(input[i])]
    return decoded

y = splice(encoded_des, False)
y1 = splice(encoded_des, pad = True)
X = splice(encoded_anc, False)

print(y1.shape)
print(X.shape)
print(X)

onehot_encoder.transform(np.ones(1).reshape(-1,1))
for i in range (100):
    print(decoder(X[i]), decoder(y1[i]), decoder(y[i]))

100%|██████████| 15/15 [00:00<00:00, 45458.50it/s]

(418404, 15, 86)
(418404, 15, 86)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0


100%|██████████| 15/15 [00:00<00:00, 24508.98it/s]

GCTCTGGCTTTGGCC 0GGTCTGGCTTTGGC GGTCTGGCTTTGGCC
CTGGGAGAGCAGGTG 0CTGGGAGAGCAGGT CTGGGAGAGCAGGTG
GAAGATCAGGCAGGC 0GAAGATCAGGCAGG GAAGATCAGGCAGGC
CATCGCTGTAGCAGA 0CATCGCTGCCACAG CATCGCTGCCACAGA
ACCCAGTGGATTGGC 0ACCCAGTGGATTGG ACCCAGTGGATTGGC
CTAGGTGGGATCTCT 0CTAGGTGGGATCTC CTAGGTGGGATCTCT
GAGCTGAACAAGCCC 0GAGCTCAACAAGCC GAGCTCAACAAGCCC
TCTCCGGGTGGTAGG 0TCTCTGGGTGGTAG TCTCTGGGTGGTAGG
TGCAGAGAGGGGAGG 0TGCAGAGACGGGAG TGCAGAGACGGGAGG
GGCAGAGCCGCAGGC 0GGCAGAGCCGCAGG GGCAGAGCCGCAGGC
ACAGCTAAGAGGGCT 0ACAGCCAAGAGGGC ACAGCCAAGAGGGCT
GAGGAAACGGTAGAA 0GAAGAAATGGTAGA GAAGAAATGGTAGAA
CGGAGCAGCTGGTGA 0CGGAGCAGCTGGTG CGGAGCAGCTGGTGA
TGTGTGGGTGACGTC 0TGTGTGG-----GC TGTGTGG-----GCC
CACCGGCCCCAGGCT 0CACCGGCCCCAGGC CACCGGCCCCAGGCT
CCTGTCTCCCCCCAG 0CCTGTCTCCCCCCA CCTGTCTCCCCCCAG
GTGTGTGGTGATGGC 0GTGTGTGGTGATGC GTGTGTGGTGATGCC
AGGCATGCCCTTCCC 0AGGCATGCCCTTCC AGGCATGCCCTTCCC
CAACATCAGGTCTCC 0CAGCATCAGGTCTC CAGCATCAGGTCTCC
TCGTTCCCTATCTGT 0TCGTTCCCTATCTG TCGTTCCCTATCTGT
CACCATTTCCTGTCG 0CACCATTTCCTGTC CACCATTT


100%|██████████| 15/15 [00:00<00:00, 29845.62it/s]


CCCTGTTTGCAAGTC 0CCCTGCTTGCAAGT CCCTGCTTGCAAGTC
CCCTGTCTGCAGCCT 0CCCTGTCTGTAGCC CCCTGTCTGTAGCCT
CATCCCTGTCACATC 0CACCCCTGTCGTAT CACCCCTGTCGTATC
CTGACTACAACAACA 0CTGACTACAATAAC CTGACTACAATAACA
GCTTCTGGGTGTCCC 0GCTTCTGGGTGTCC GCTTCTGGGTGTCCC
TGGCATCCACTCTCT 0TGGCATCCACTCTC TGGCATCCACTCTCT
CTCCCTTCTTGTCCC 0CTCCCTTCTTGTCC CTCCCTTCTTGTCCC
TTCCGTGACGGATGC 0TTCCGTGACGGATG TTCCGTGACGGATGC
CTGATGAACCTTCCC 0CTGAGGAACCTTCC CTGAGGAACCTTCCC
CAAACTCTTCTGTCC 0CAAACTCTTCTGTC CAAACTCTTCTGTCC
CATCCCTGCCCTGCT 0CATCCCTGCCCTGC CATCCCTGCCCTGCT
CAAAATCCAATCACA 0CAAAATCCAATCAC CAAAATCCAATCACA
GCTCCCTAACACTCC 0GCTCCCTAACACGC GCTCCCTAACACGCC
TGAATCAAACTTGAA 0TGAATC-AACTTGA TGAATC-AACTTGAA
GTCCTGTCCTGAGTA 0GTCCTGTCTTGAGT GTCCTGTCTTGAGTA
CTCCGTGGGCCCTAA 0ATCCGTGGGCCCTA ATCCGTGGGCCCTAA
CTCACTCATCCCAAC 0CTCACTCATCCCAA CTCACTCATCCCAAC
TCTTCACTCACTGCC 0TCTTCACTCACTGC TCTTCACTCACTGCC
TTGCCCCACACCCTG 0CTGCCCCACACCCT CTGCCCCACACCCTG
CCAGGGAGCCTCCCA 0CCAGGGAGCCTCCC CCAGGGAGCCTCCCG
TGGCACCGTGGGGAC 0TGGCACCGTGGGGA TGGCACCG

100%|██████████| 15/15 [00:00<00:00, 42625.04it/s]


CTGGCCCACAGGCTC 0CTGGCCCACAGGCT CTGGCCCACAGGCTC
ACGGGAAGTCAGCCT 0ACGGAAAGTCAGCC ACGGAAAGTCAGCCT
CTCATGCCCTGAGAG 0CTCATGCCCCGAGA CTCATGCCCCGAGAG
CTGAGTGCAAGGGAG 0CTGAGTGCAAGGGA CTGAGTGCAAGGGAG
AGGCAGCGCTGTCTG 0AGGCAGCGCTGTCT AGGCAGCGCTGTCTG
TGCTTCCCATGCAGA 0TGCTTCCCATGCAG TGCTTCCCATGCAGA
AGCACCCCCCTCCCA 0AGCACCCCCCTCCC AGCACCCCCCTCCCA
CCCCTGTGCAGGCCG 0CCCCTGTGCAGGCC CCCCTGTGCAGGCCG
GCCTTCCCGGCAGAC 0GCCTTCGCGGCAGA GCCTTCGCGGCAGAC
CACCATACACCACGT 0CACCATACACCACG CACCATACACCACGT
TCCAAGCCACACTGA 0TCCAAGCCACACTG TCCAAGCCACACTGA
GGCTTCTCTCCAAGC 0GGCCTCCCTCCAAG GGCCTCCCTCCAAGC
CTGCGGCCCCCATTT 0CTGCAGCCCCCATT CTGCAGCCCCCATTT
CTAGACCCCACCAGG 0CCAGACCCTGCCAG CCAGACCCTGCCAGG
GCGACCTGCACACCC 0GCAACCTGCATATC GCAACCTGCATATCC
ACCTCCCTACCCTGC 0ACCTCCCTACCCTG ACCTCCCTACCCTGC
CCCCCTCCTCCAGGA 0CCCCCTCTTCCAGG CCCCCTCTTCCAGGA
GCCTGCCCTATGTGG 0GTCTGCCCTATGTG GTCTGCCCTATGTGG
AGTAGGCACGTGGTT 0AGTAAGCACGTGGT AGTAAGCACGTGGTT
TTCCTCTCCAGCAAC 0TTCCTCTTCAGCAA TTCCTCTTCAGCAAC
TATTTCCTTTTTACT 0TATTTCCTTTTTAC TATTTCCT

100%|██████████| 15/15 [00:00<00:00, 41887.19it/s]

GCTTAGTCCCAGAGC 0GCTTAGTCCCAGAG GCTTAGTCCCAGAGC
TTCAGGTGGGGCTGC 0TTCAGGTGGGGCTG TTCAGGTGGGGCTGC
CCACAGAGCTCCTCA 0CCACAGAGCTCCTC CCACAGAGCTCCTCA
GTCTAAGCCAAGTGG 0GTCTAAGCCAAGTG GTCTAAGCCAAGTGG


In [7]:
X_train,  y_train1,  y_train, = X, y1, y 

def concat(input1, input2):
    result = []
    for x, y in tqdm(zip(input1, input2)):
        result.append(np.concatenate((x, y), axis=1))
        # print(decoder(x), decoder(y), decoder(np.concatenate((x, y))))
    
    return np.array(result)

y_train1 = concat(X_train, y_train1)
#y_val1 = concatenate(X_val, y_val1)
# y_test1 = concat(X_test, y_test1)

# for i in range (100):
#     print(decoder(y_train[i]))
    
nucleotide = label_encoder.inverse_transform(range(encode_dimension))

def decoder(input):
    nucleotide = label_encoder.inverse_transform(range(encode_dimension))
    decoded = ''
    for i in tqdm(range(len(input))):
        # decoded= decoded+(nucleotide[np.argmax(onehot_encoder.inverse_transform(input[i].reshape(-1, 1)))])
        #print(np.argmax(input[i]))
        #print(nucleotide)
        decoded= decoded+nucleotide[np.argmax(input[i])]
    return decoded

418404it [00:05, 75797.24it/s] 


In [8]:
onehot_encoder.transform(np.ones(1).reshape(-1,1))
for i in range (100):
    print(decoder(X_train[i]), decoder(y_train1[i]), decoder(y_train[i]))
print(y_train1[1])

  0%|          | 0/15 [00:00<?, ?it/s]

TCTGACTTCCAGCAA TCTGACTTCCAGCAA TCTGACTTCCAGCAA
CTGCTGGCCTGTGCC CTGCTGGCCTGTGCC CTGCTGGCCTGTGCC
AGGGTGGAAGCTGAG AGGGTGGAAGCTGAG AGGGTGCAAGCTGAG
CACTGGAGTGGAGTT CACTGGAGTGGAGTT CACTGGAGTGGAGTT
TTCCTGTGGAGAGGA TTCCTGTGGAGAGGA TTCCTGTGGAGAGGA
GCCATGCCCAGAGTG GCCATGCCCAGAGTG GCCATGCCTAGAGTG
GGATGGGCCAGGTGT GGATGGGCCAGGTGT GGATGGGCCAT-TGT
TCATATTCTGGCCCC TCATATTCTGGCCCC TCATCTTCTGGCCCC
TGTTGTCTGCATGTA TGTTGTCTGCATGTA TGTTGTCTGCATGTA
ACCTCATACCACAAC ACCTCATACCACAAC ACTTAATACCACAAC
CAGGCATGGGGGGGC CAGGCATGGGGGGGC CAGGCATAGGGGGAT
TGGAGGAAAGATTGG TGGAGGAAAGATTGG TGGAGGAAAGAT---
AGAAAAGATGAGTGA AGAAAAGATGAGTGA ---------GAGTGA
GAGGATCAACTTCTC GAGGATCAACTTCTC GAGCATCAACTTCTC
TGACAACCTAGGCCA TGACAACCTAGGCCA TCACAACCTAGGCCA
GTAAGTAGTGCTTCT GTAAGTAGTGCTTCT GTAAGTAGTGCTTGT
GCTCATCTCCTTGGC GCTCATCTCCTTGGC GCTCATCTCCTTGGC
TGTGATACGCGGCCG TGTGATACGCGGCCG TGTGATACGTGGCCG
GCCCTCGCTCCAGCA GCCCTCGCTCCAGCA GCCCTCGCTCCAGCA
GCTGGACCCCTACCT GCTGGACCCCTACCT GCTGGACCCCTACCT
GCCGTCTGCTGCCAT GCCGTCTGCTGCCAT GCCGTCTG

100%|██████████| 15/15 [00:00<00:00, 44431.19it/s]


GAAGATCAGGCAGGC GAAGATCAGGCAGGC GAAGATCAGGCAGGC
CATCGCTGTAGCAGA CATCGCTGTAGCAGA CATCGCTGCCACAGA
ACCCAGTGGATTGGC ACCCAGTGGATTGGC ACCCAGTGGATTGGC
CTAGGTGGGATCTCT CTAGGTGGGATCTCT CTAGGTGGGATCTCT
GAGCTGAACAAGCCC GAGCTGAACAAGCCC GAGCTCAACAAGCCC
TCTCCGGGTGGTAGG TCTCCGGGTGGTAGG TCTCTGGGTGGTAGG
TGCAGAGAGGGGAGG TGCAGAGAGGGGAGG TGCAGAGACGGGAGG
GGCAGAGCCGCAGGC GGCAGAGCCGCAGGC GGCAGAGCCGCAGGC
ACAGCTAAGAGGGCT ACAGCTAAGAGGGCT ACAGCCAAGAGGGCT
GAGGAAACGGTAGAA GAGGAAACGGTAGAA GAAGAAATGGTAGAA
CGGAGCAGCTGGTGA CGGAGCAGCTGGTGA CGGAGCAGCTGGTGA
TGTGTGGGTGACGTC TGTGTGGGTGACGTC TGTGTGG-----GCC
CACCGGCCCCAGGCT CACCGGCCCCAGGCT CACCGGCCCCAGGCT
CCTGTCTCCCCCCAG CCTGTCTCCCCCCAG CCTGTCTCCCCCCAG
GTGTGTGGTGATGGC GTGTGTGGTGATGGC GTGTGTGGTGATGCC
AGGCATGCCCTTCCC AGGCATGCCCTTCCC AGGCATGCCCTTCCC
CAACATCAGGTCTCC CAACATCAGGTCTCC CAGCATCAGGTCTCC
TCGTTCCCTATCTGT TCGTTCCCTATCTGT TCGTTCCCTATCTGT
CACCATTTCCTGTCG CACCATTTCCTGTCG CACCATTTCCTGTCG
TCGTTTCCTCTGAAT TCGTTTCCTCTGAAT TCGTTTCCTCTGAAT
GTCTCACCCTGCCCT GTCTCACCCTGCCCT GTCTCACC

100%|██████████| 15/15 [00:00<00:00, 51653.99it/s]


CCCTGTCTGCAGCCT CCCTGTCTGCAGCCT CCCTGTCTGTAGCCT
CATCCCTGTCACATC CATCCCTGTCACATC CACCCCTGTCGTATC
CTGACTACAACAACA CTGACTACAACAACA CTGACTACAATAACA
GCTTCTGGGTGTCCC GCTTCTGGGTGTCCC GCTTCTGGGTGTCCC
TGGCATCCACTCTCT TGGCATCCACTCTCT TGGCATCCACTCTCT
CTCCCTTCTTGTCCC CTCCCTTCTTGTCCC CTCCCTTCTTGTCCC
TTCCGTGACGGATGC TTCCGTGACGGATGC TTCCGTGACGGATGC
CTGATGAACCTTCCC CTGATGAACCTTCCC CTGAGGAACCTTCCC
CAAACTCTTCTGTCC CAAACTCTTCTGTCC CAAACTCTTCTGTCC
CATCCCTGCCCTGCT CATCCCTGCCCTGCT CATCCCTGCCCTGCT
CAAAATCCAATCACA CAAAATCCAATCACA CAAAATCCAATCACA
GCTCCCTAACACTCC GCTCCCTAACACTCC GCTCCCTAACACGCC
TGAATCAAACTTGAA TGAATCAAACTTGAA TGAATC-AACTTGAA
GTCCTGTCCTGAGTA GTCCTGTCCTGAGTA GTCCTGTCTTGAGTA
CTCCGTGGGCCCTAA CTCCGTGGGCCCTAA ATCCGTGGGCCCTAA
CTCACTCATCCCAAC CTCACTCATCCCAAC CTCACTCATCCCAAC
TCTTCACTCACTGCC TCTTCACTCACTGCC TCTTCACTCACTGCC
TTGCCCCACACCCTG TTGCCCCACACCCTG CTGCCCCACACCCTG
CCAGGGAGCCTCCCA CCAGGGAGCCTCCCA CCAGGGAGCCTCCCG
TGGCACCGTGGGGAC TGGCACCGTGGGGAC TGGCACCGTGGGGAC
ACAAAGGAACCAAGG ACAAAGGAACCAAGG ACAAAGGA

  0%|          | 0/15 [00:00<?, ?it/s]

CTGGCCCACAGGCTC CTGGCCCACAGGCTC CTGGCCCACAGGCTC
ACGGGAAGTCAGCCT ACGGGAAGTCAGCCT ACGGAAAGTCAGCCT
CTCATGCCCTGAGAG CTCATGCCCTGAGAG CTCATGCCCCGAGAG
CTGAGTGCAAGGGAG CTGAGTGCAAGGGAG CTGAGTGCAAGGGAG
AGGCAGCGCTGTCTG AGGCAGCGCTGTCTG AGGCAGCGCTGTCTG
TGCTTCCCATGCAGA TGCTTCCCATGCAGA TGCTTCCCATGCAGA
AGCACCCCCCTCCCA AGCACCCCCCTCCCA AGCACCCCCCTCCCA
CCCCTGTGCAGGCCG CCCCTGTGCAGGCCG CCCCTGTGCAGGCCG
GCCTTCCCGGCAGAC GCCTTCCCGGCAGAC GCCTTCGCGGCAGAC
CACCATACACCACGT CACCATACACCACGT CACCATACACCACGT
TCCAAGCCACACTGA TCCAAGCCACACTGA TCCAAGCCACACTGA
GGCTTCTCTCCAAGC GGCTTCTCTCCAAGC GGCCTCCCTCCAAGC
CTGCGGCCCCCATTT CTGCGGCCCCCATTT CTGCAGCCCCCATTT
CTAGACCCCACCAGG CTAGACCCCACCAGG CCAGACCCTGCCAGG
GCGACCTGCACACCC GCGACCTGCACACCC GCAACCTGCATATCC
ACCTCCCTACCCTGC ACCTCCCTACCCTGC ACCTCCCTACCCTGC
CCCCCTCCTCCAGGA CCCCCTCCTCCAGGA CCCCCTCTTCCAGGA
GCCTGCCCTATGTGG GCCTGCCCTATGTGG GTCTGCCCTATGTGG
AGTAGGCACGTGGTT AGTAGGCACGTGGTT AGTAAGCACGTGGTT
TTCCTCTCCAGCAAC TTCCTCTCCAGCAAC TTCCTCTTCAGCAAC
TATTTCCTTTTTACT TATTTCCTTTTTACT TATTTCCT

100%|██████████| 15/15 [00:00<00:00, 28873.13it/s]

GCTTAGTCCCAGAGC GCTTAGTCCCAGAGC GCTTAGTCCCAGAGC
TTCAGGTGGGGCTGC TTCAGGTGGGGCTGC TTCAGGTGGGGCTGC
CCACAGAGCTCCTCA CCACAGAGCTCCTCA CCACAGAGCTCCTCA
GTCTAAGCCAAGTGG GTCTAAGCCAAGTGG GTCTAAGCCAAGTGG
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
def printHitMiss(a,b):
    if a==b:
        return 'Hit'
    else:
        return 'Miss'
def accuracy(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count = count+1
    return count/len(a)

def accuracy2(a, b, c):
    count = 0
    count2 =0
    for i in range(len(a)):
        if a[i] != c[i]:
            count2 = count2 +1
        if a[i] != c[i] and b[i]==c[i]:
            count = count+1
    return count/count2

def isMutation(a, b):
    if a!= b:
        print("mutation")

In [12]:
# In[ ]:

def lstm_model(latent_dim, half):
    batch_size = 1000  # Batch size for training.
    epochs = 45  # Number of epochs to train for.
#     latent_dim = 128  # Latent dimensionality of the encoding space.
#     half = 64
    num_samples = 10000  # Number of samples to train on.
    encoder_inputs = layers.Input(shape=(None, encode_dimension))
    
    encoder = layers.Bidirectional(layers.LSTM(half, return_state=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
    state_h = layers.concatenate([forward_h, backward_h])
    state_c = layers.concatenate([forward_c, backward_c])
    
    
    # only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder
    decoder_inputs = layers.Input(shape=(None, 2*encode_dimension))
    decoder_lstm = layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = layers.Dense(encode_dimension, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # inference
    encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = layers.Input(shape=(latent_dim,))
    decoder_state_input_c = layers.Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

    # Run training
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']
                  )
    return model, encoder_model, decoder_model


def modelFit(epoch, batchSize, latent_dim, half, X_train, y_train, y_train1):
    model1, encoder_model1, decoder_model1 = lstm_model(latent_dim, half)
    hist1 = model1.fit([X_train, y_train1], y_train,
          batch_size=batchSize,
          epochs=epoch,
          #validation_data=([X_test,y_test1], y_test),
          validation_split=0.2,
          verbose = 1
         )
    return hist1, model1, encoder_model1, decoder_model1

def grid_search(latent, half,train_size, X_train, y_train, y_train1):
#     hist1, model1, encoder_model1, decoder_model1 = modelFit(1, 256, latent, half, X_train, y_train, y_train1)
#     hist2 ,model2, encoder_model2, decoder_model2 = modelFit(2, 256, latent, half, X_train, y_train, y_train1)
    hist3 ,model3, encoder_model3, decoder_model3 = modelFit(15, 256, latent, half, X_train, y_train, y_train1)
    #hist4 ,model4, encoder_model4, decoder_model4 = modelFit(30, 1000, latent, half, X_train, y_train, y_train1)
    #hist5 ,model5, encoder_model5, decoder_model5 = modelFit(50, 100, latent, half, X_train, y_train, y_train1)
    #hist6 ,model6, encoder_model6, decoder_model6 = modelFit(80, 100, latent, half, X_train, y_train, y_train1)
    #hist7 ,model7, encoder_model7, decoder_model7 = modelFit(100, 100, latent, half, X_train, y_train, y_train1)
    #hist8 ,model8, encoder_model8, decoder_model8 = modelFit(500, 100, latent, half)

#     model1.save("models/insert2_{}_{}_{}_{}_1.h5".format(ancName, desName,ancName, desName))
#     model2.save("models/insert2_{}_{}_{}_{}_2.h5".format(ancName, desName,ancName, desName))
    model3.save("models/insert2_{}_{}_{}_{}_{}_10.h5".format(ancName, desName,ancName, desName,strand))
    #model4.save("models/insert2_{}_{}_30_double.h5".format(train_size,half))
    #model5.save("models/_gap_hg38_{}_{}_50_double.h5".format(train_size,half))
    #model6.save("models/_gap_hg38_{}_{}_80_double.h5".format(train_size,half))
    #model7.save("models/_gap_hg38_{}_{}_100_double.h5".format(train_size,half))
    #model8.save("_gap_hg38_{}_{}_500.h5".format(train_size,half))
    
#     encoder_model1.save("models/E_insert2_{}_{}_{}_{}_1.h5".format(ancName, desName,ancName, desName))
#     encoder_model2.save("models/E_insert2_{}_{}_{}_{}_2.h5".format(ancName, desName,ancName, desName))
    encoder_model3.save("models/E_insert2_{}_{}_{}_{}_{}_10.h5".format(ancName, desName,ancName, desName,strand))
    #encoder_model4.save("models/E_insert2_{}_{}_30_double.h5".format(train_size,half))
    #encoder_model5.save("models/E_gap_hg38_{}_{}_50_double.h5".format(train_size,half))
    #encoder_model6.save("models/E_gap_hg38_{}_{}_80_double.h5".format(train_size,half))
    #encoder_model7.save("models/E_gap_hg38_{}_{}_100_double.h5".format(train_size,half))
    #encoder_model8.save("E_gap_hg38_{}_{}_500.h5".format(train_size,half))
    
#     decoder_model1.save("models/D_insert2_{}_{}_{}_{}_1.h5".format(ancName, desName,ancName, desName))
#     decoder_model2.save("models/D_insert2_{}_{}_{}_{}_2.h5".format(ancName, desName,ancName, desName))
    decoder_model3.save("models/D_insert2_{}_{}_{}_{}_{}_10.h5".format(ancName, desName,ancName, desName,strand))
    #decoder_model4.save("models/D_insert2_{}_{}_30_double.h5".format(train_size,half)),strand
    #decoder_model5.save("models/D_gap_hg38_{}_{}_50_double.h5".format(train_size,half))
    #decoder_model6.save("models/D_gap_hg38_{}_{}_80_double.h5".format(train_size,half))
    #decoder_model7.save("models/D_gap_hg38_{}_{}_100_double.h5".format(train_size,half))
    #decoder_model8.save("D_gap_hg38_{}_{}_500.h5".format(train_size,half))
    
#     count = [i for i in range(len(hist3.history['val_loss']))]
#     val_loss_hist.append([hist3.history['val_loss'].index(min(hist3.history['val_loss'])),min(hist3.history['val_loss'])])
#     print(val_loss_hist)
#     for i, value in zip(count, hist3.history['val_loss']):
#         print(i, value)

# grid_search(2, 1, 000, X_train, y_train, y_train1)
# grid_search(16, 8, 000, X_train, y_train, y_train1)        
# grid_search(32, 16, 000, X_train, y_train, y_train1)
# grid_search(64, 32, 000, X_train, y_train, y_train1)
# grid_search(128, 64, 000, X_train, y_train, y_train1)
# grid_search(256, 128, 000, X_train, y_train, y_train1)
# grid_search(512, 256, 000, X_train, y_train, y_train1)
grid_search(1024, 512, 000, X_train, y_train, y_train1)
#grid_search(8192, 4096, 000, X_train, y_train, y_train1)

with open('loss_hist.txt', 'wb') as fp:
    pickle.dump(val_loss_hist, fp)

Train on 334723 samples, validate on 83681 samples
Epoch 1/15
334723/334723 [==============================] - 49s 147us/sample - loss: 0.2881 - accuracy: 0.9434 - val_loss: 0.1794 - val_accuracy: 0.9654
Epoch 2/15
334723/334723 [==============================] - 43s 130us/sample - loss: 0.1940 - accuracy: 0.9617 - val_loss: 0.1764 - val_accuracy: 0.9654
Epoch 3/15
334723/334723 [==============================] - 44s 131us/sample - loss: 0.1913 - accuracy: 0.9618 - val_loss: 0.1748 - val_accuracy: 0.9655
Epoch 4/15
334723/334723 [==============================] - 44s 130us/sample - loss: 0.1903 - accuracy: 0.9618 - val_loss: 0.1750 - val_accuracy: 0.9655
Epoch 5/15
334723/334723 [==============================] - 44s 131us/sample - loss: 0.1895 - accuracy: 0.9618 - val_loss: 0.1742 - val_accuracy: 0.9655
Epoch 6/15
334723/334723 [==============================] - 44s 130us/sample - loss: 0.1891 - accuracy: 0.9619 - val_loss: 0.1744 - val_accuracy: 0.9655
Epoch 7/15
334723/334723 [=====

KeyboardInterrupt: 

In [11]:
%precision 2
seq_length = 11
# test_size = len(y_test)
val_size = 30000

# key = ['-', '0', 'A', 'B', 'C', 'G', 'I', 'L', 'N', 'O', 'P', 'T', 'V' ,'X' ,'b', 'c', 'f', 'g', 'h', 'i',
#        'o', 'p' ,'r']

mapDict = {'A': 'A', 'C': 'C', 'G': 'G', 'T': 'T', '-': '-', 'AAA': 'U', 'AAC': '\x0e', 'AAT': '\x03', 
           'AAG': 'b', 'ACA': '#', 'ACC': '\x1f', 'ACT': '\x1c', 'ACG': '\x11', 'ATA': '|', 'ATC': '\r',
           'ATT': 'Z', 'ATG': 'w', 'AGA': 'H', 'AGC': '\x0b', 'AGT': 'N', 'AGG': '6', 'CAA': '\x04', 
           'CAC': '}', 'CAT': 'r', 'CAG': '\x1b', 'CCA': '\x1d', 'CCC': '@', 'CCT': 'P', 'CCG': 'n', 'CTA': 'J',
           'CTC': '\x19', 'CTT': '_', 'CTG': 'W', 'CGA': ']', 'CGC': 's', 'CGT': '5', 'CGG': 'y', 'TAA': '9', 
           'TAC': 'q', 'TAT': '{', 'TAG': '\x00', 'TCA': '\x14', 'TCC': 'p', 'TCT': '+', 'TCG': 'c', 'TTA': '\x13',
           'TTC': 'l', 'TTT': 'Y', 'TTG': 'v', 'TGA': 'm', 'TGC': '0', 'TGT': '\x0c', 'TGG': '-', 'GAA': ',', 
           'GAC': '!', 'GAT': '\x05', 'GAG': ':', 'GCA': 'F', 'GCC': '\x0f', 'GCT': 'Q', 'GCG': '\n', 'GTA': '%', 
           'GTC': '.', 'GTT': '\x18', 'GTG': '\x08', 'GGA': 'L', 'GGC': '*', 'GGT': '~', 'GGG': '1', 'AA': '\x12', 
           'AC': '?', 'AT': 'u', 'AG': '=', 'CA': '7', 'CC': '\x06', 'CT': 'B',
           'CG': 'x', 'TA': 'k', 'TC': '(', 'TT': '\x17', 'TG': 'g', 'GA': '<', 'GC': '\x16', 'GT': 'S', 'GG': '>'}

rev_dict = {v: k for k, v in mapDict.items()}
#print(rev_dict.keys())
rev_key = []
for item in key:
    #print(item)
    if item in list(rev_dict.keys()):
        rev_key.append(rev_dict[item])
        #print('hi')
    else :
        rev_key.append(item)
print(rev_key)

nucleotide = label_encoder.inverse_transform(range(encode_dimension))

def printHitMiss(a,b):
    if a==b:
        return 'Hit'
    else:
        return 'Miss'
    
def accuracy(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count = count+1
    return count/len(a)

def accuracy2(a, b, c):
    count = 0
    count2 =0
    for i in range(len(a)):
        if a[i] != c[i]:
            count2 = count2 +1
        if a[i] != c[i] and b[i]==c[i]:
            count = count+1
    return count/count2

def isMutation(a, b):
    if a!= b:
        print("mutation")


def decode_sequence(input_seq, model, encoder_model, decoder_model):
    nucleotide = label_encoder.inverse_transform(range(encode_dimension))
    # Encode the input as state vectors.
    #print(input_seq[0,0])
    index = 0
    states_value = encoder_model.predict(input_seq)
    #print(len(states_value))
    #print(states_value)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, encode_dimension*2))
    target_seq[0][0]= np.hstack((input_seq[0,index], onehot_encoder.transform(np.ones(1).reshape(-1,1))[0]))
    #print(target_seq)
    # Populate the first character of target sequence with the start character.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_seq = ''
    probability = 1
    
    while not stop_condition:
        index = index +1
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.random.choice(encode_dimension, 1, p=output_tokens[0, -1, :])[0]
        #list(mapDict.keys())
        for i in range(len(output_tokens[0])):
            print(i, dict(zip(rev_key, output_tokens[0][i])))
        sampled_nucleotide = nucleotide[np.random.choice(encode_dimension, 1, p=output_tokens[0, -1, :])[0]]
        
        decoded_seq += sampled_nucleotide
        #print(sampled_nucleotide, decoded_seq)
        #print(decoded_sentence)
        # Exit condition: either hit max length
        # or find stop character.
        if (len(decoded_seq) == seq_length):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, encode_dimension*2))
        temp = np.zeros((encode_dimension))
        temp[sampled_token_index] = 1
        target_seq[0][0]= np.hstack((input_seq[0, index], temp))
        # target_seq[0, 0, sampled_token_index] = 1
        
        
        # Update states
        states_value = [h, c]

    return decoded_seq

def get_prob(input_seq, target, model, encoder_model, decoder_model):
    # Encode the input as state vectors.
    nucleotide = label_encoder.inverse_transform(range(encode_dimension))
    index = 0
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, encode_dimension*2))
    target_seq[0][0]= np.hstack((input_seq[0,index], onehot_encoder.transform(np.ones(1).reshape(-1,1))[0]))

    stop_condition = False
    decoded_seq = ''
    probability = []
    
    while not stop_condition:
        index = index +1
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(target[index-1])
        
        probability.append(output_tokens[0, -1, :][sampled_token_index])
        sampled_nucleotide = nucleotide[np.random.choice(encode_dimension, 1, p=output_tokens[0, -1, :])[0]]
        
        decoded_seq += sampled_nucleotide
        if (len(decoded_seq) == seq_length):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, encode_dimension*2))
        temp = np.zeros((encode_dimension))
        temp[sampled_token_index] = 1
        target_seq[0][0]= np.hstack((input_seq[0, index], temp))
        # target_seq[0, 0, sampled_token_index] = 1
        
        
        # Update states
        states_value = [h, c]

    return decoded_seq, probability

def diffList(a, b):
    count = 0
    length = len(a)
    for i in range(length):
        if a[i] != b[i]:
            count = count+1
    return count

def decoder(input):
    nucleotide = label_encoder.inverse_transform(range(encode_dimension))
    decoded = ''
    for i in range(len(input)):
        # decoded= decoded+(nucleotide[np.argmax(onehot_encoder.inverse_transform(input[i].reshape(-1, 1)))])
        #print(np.argmax(input[i]))
        #print(nucleotide)
        decoded= decoded+nucleotide[np.argmax(input[i])]
    return decoded

#for seq_index in range(1):
def predict2(X_test, y_test, model, encoder_model, decoder_model, gru=False):
    x_true =[]
    y_hat =[]
    y_true =[]
    probList=[]
    generator_output = []
    productProb = [0]*seq_length
    for seq_index in range(len(X_test)):
        input_seq = X_test[seq_index: seq_index + 1]
        if gru:
            decoded_sentence = decode_gru(input_seq, model, encoder_model, decoder_model)
        else :
            decoded_sentence = decode_sequence(input_seq, model, encoder_model, decoder_model)
        _, prob = get_prob(input_seq, y_test[seq_index], model, encoder_model, decoder_model)
        probList.append(prob)
        prob = [math.log(x) for x in prob]
        productProb = [sum(x) for x in zip(productProb, prob)]
        input_sen = decoder(input_seq[0])
        print(input_sen, ' -> ',
              decoded_sentence, 'True:', decoder(y_test[seq_index]), 
              printHitMiss(decoded_sentence, decoder(y_test[seq_index]))
              #diffList(input_sen, decoded_sentence)
             )
        print(input_sen, ' -> ',
              decoder(y_test[seq_index]), 'True:', decoder(y_test[seq_index]), 
              prob,
              printHitMiss(decoded_sentence, decoder(y_test[seq_index]))
              #diffList(input_sen, decoded_sentence)
             )
        x_true.append(input_sen)
        y_hat.append(decoded_sentence)
        y_true.append(decoder(y_test[seq_index]))
    #generator_output.append(input_sen+decoded_seq)
    print(productProb)
    productProb = [x/test_size for x in productProb]
    print("Mean and std of probabilities : {} , {}  ".format(np.mean(probList), np.std(probList)))
    print("Sum of log probabilities : {}".format(productProb))
    print("Percentage of target and prediction being identical: {}".format(accuracy(y_hat, y_true)))
    print("Percentage of training and prediction being identical: {}".format(accuracy(y_hat, x_true)))
    print("Accuracy given mutation happened : {}".format(accuracy2(x_true, y_hat, y_true)))
    #np.save('data/hg38_output.npy', generator_output)

def grid_predict(train_size, half, epoch, X_test, y_test):
    model1 = load_model("models/insert2_{}_{}_{}.h5".format(train_size,half,epoch))

    encoder_model1 = load_model("models/E_insert2_{}_{}_{}.h5".format(train_size,half, epoch))

    decoder_model1 =load_model("models/D_insert2_{}_{}_{}.h5".format(train_size,half, epoch))

    predict2(X_test, y_test, model1, encoder_model1, decoder_model1, gru=False)
    



# In[3]:
def concat(input1, input2):
    result = []
    for x, y in zip(input1, input2):
        result.append(np.hstack((x, y)))
    
    return np.array(result)

# def get_data(trainInd, valInd, testInd):
#     X_train=np.load('prepData/X_train_gap_hg38_v3_chr2_size10.npy')[:trainInd]
#     X_val=np.load('prepData/X_val_gap_hg38_v3_chr2_size10.npy')[:valInd]
#     X_test=np.load('prepData/X_test_gap_hg38_v3_chr2_size10.npy')[:testInd]
#     y_train=np.load('prepData/y_train_gap_hg38_v3_chr2_size10.npy')[:trainInd]
#     y_val=np.load('prepData/y_val_gap_hg38_v3_chr2_size10.npy')[:valInd]
#     y_test=np.load('prepData/y_test_gap_hg38_v3_chr2_size10.npy')[:testInd]

#     y_train1 = np.load('prepData/y_train1_gap_hg38_v3_chr2_size10.npy')[:trainInd]
#     y_val1 = np.load('prepData/y_val1_gap_hg38_v3_chr2_size10.npy')[:valInd]
#     y_test1 = np.load('prepData/y_test1_gap_hg38_v3_chr2_size10.npy')[:testInd]

#     y_train1 = concat(X_train, y_train1)
#     y_val1 = concat(X_val, y_val1)
#     y_test1 = concat(X_test, y_test1)
#     return X_test, y_test




train_size = 0
# hidden = [16, 32, 64,128,256,512]
# epoch = [10, 10, 2, 2, 2, 1]
hidden = [256]
epoch = [10]
#X_test, y_test = get_data(train_size, val_size, test_size)
for h, e in zip(hidden, epoch):
    print("Train size = {}, hidden_size = {}, epoch = {}".format(train_size, h, e))
    grid_predict(train_size, h, e, X_test, y_test)
    print("The end of Train size = {}, hidden_size = {}, epoch = {}".format(train_size, h, e))
    

# mapDict = {'A': 'A', 'C': 'C', 'G': 'G', 'T': 'T', '-': '-', 'AA': 'O', 'AC': 'h', '0': '0',
#        'AT': 'b', 'AG': 'V', 'CA': 'r', 'CC': 'p', 'CT': 'o', 'CG': 'i', 'TA': 'g', 
#        'TC': 'I', 'TT': 'f', 'TG': 'L', 'GA': 'B', 'GC': 'c', 'GT': 'X', 'GG': 'P'}

['GAC', '"', '$', 'GTA', '&', 'TC', ')', 'GGC', 'TCT', 'TGG', 'GTC', '/', 'TGC', 'GGG', '2', '3', '4', 'CGT', 'AGG', 'CA', '8', 'TAA', 'GAG', ';', 'GA', 'AG', 'GG', 'AC', 'CCC', 'A', 'CT', 'C', 'D', 'E', 'GCA', 'G', 'AGA', 'I', 'CTA', 'K', 'GGA', 'M', 'AGT', 'CCT', 'GCT', 'R', 'GT', 'T', 'AAA', 'V', 'CTG', 'X', 'TTT', 'ATT', '[', '\\', 'CGA', '^', 'CTT', '`', 'a', 'AAG', 'TCG', 'd', 'e', 'f', 'TG', 'h', 'i', 'j', 'TTC', 'TGA', 'CCG', 'o', 'TCC', 'TAC', 'CGC', 't', 'AT', 'ATG', 'CG', 'z', 'TAT', 'ATA', 'CAC', 'GGT']
Train size = 0, hidden_size = 256, epoch = 10


NameError: name 'X_test' is not defined

In [ ]:
onehot_encoder.transform(np.ones(1).reshape(-1,1))
for i in range (100):
    print(decoder(X_train[i]), decoder(y_train[i]), decoder(y_train1[i]))
    
print(y_train[1].shape)